In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import mean_squared_error

In [4]:
X_train = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_train_27022019.csv")
y_train = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_train_27022019.csv")
X_dev = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_dev_27022019.csv")
y_dev = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_dev_27022019.csv")
X_test = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_test_27022019.csv")
y_test = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_test_27022019.csv")


In [5]:
X_dev['IsOrganic'] = X_dev['IsOrganic'].astype('int')

In [6]:
X_train['IsOrganic'] = X_train['IsOrganic'].astype('int')

In [7]:
X_test['IsOrganic'] = X_test['IsOrganic'].astype('int')

In [8]:
X_dev['order_HourGroup'] =X_dev['order_HourGroup'].astype('category')


In [9]:
X_train['order_HourGroup'] =X_train['order_HourGroup'].astype('category')

In [10]:
X_test['order_HourGroup'] =X_test['order_HourGroup'].astype('category')

In [11]:
def one_hot_encoding(data,var,drop=True):
    #res = []
    #nm = data[var].name
    nn = data[var].unique()
    for i in nn:
        s = '%s_%s' % (var,i)
        data[s] = 0
        data.loc[data[var]==i,s] = 1
        #res.append(newx)
    if(drop==True):
        data = data.drop(var,axis=1)
    return(data)

In [12]:
X_train = one_hot_encoding(X_train,'order_HourGroup')

X_dev = one_hot_encoding(X_dev,'order_HourGroup')

X_test = one_hot_encoding(X_test,'order_HourGroup')


# The following function calculates the Hamming score that calculates the multilabel accuracy. 

In [13]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)



## We selected the folowing models because they received  the most highest accuracy  therfore we will run X_test on this models
 

Support multilabel:

- sklearn.tree.DecisionTreeClassifier
- sklearn.tree.ExtraTreeClassifier
- sklearn.ensemble.ExtraTreesClassifier
- sklearn.ensemble.RandomForestClassifier
- sklearn.neighbors.KNeighborsClassifier


In [23]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.linear_model import LogisticRegression


### TRANSFORMED THE MULTILEVEL-Y TO A NUMPY ARRAY

In [14]:
y_dev.head()
ydev = y_dev.copy()
ydev = np.array(ydev)
ydev = np.delete(ydev, 0, 1) 
print(ydev.shape)
print(ydev)

(402512, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [15]:
y_test.head()
ytest = y_test.copy()
ytest = np.array(ytest)
ytest = np.delete(ytest, 0, 1) 
print(ytest.shape)
print(ytest)

(402512, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
y_train.head()
ytrain = y_train.copy()
ytrain = np.array(ytrain)
ytrain = np.delete(ytrain, 0, 1) 
print(ytrain.shape)
print(ytrain)

(3220100, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# deleting the first column in X_dev/train/test, because we can't put the user_id in the models

In [17]:

Xdev = X_dev.copy()
Xdev = Xdev.drop(['Unnamed: 0'],axis=1)
print(Xdev.shape)
Xdev.head()

(402512, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_18-24,order_HourGroup_06-12,order_HourGroup_00-06,order_HourGroup_12-18
0,1,0.024617,0.929434,1,1,0.0625,1,0,0,0
1,1,0.020455,0.929434,1,1,0.1875,0,1,0,0
2,1,0.115153,0.929434,0,2,0.2500,0,0,1,0
3,1,0.115153,0.929434,0,4,0.0625,0,0,0,1
4,1,0.064592,0.929434,1,4,0.0625,1,0,0,0


In [18]:

Xtrain = X_train.copy()
Xtrain = Xtrain.drop(['Unnamed: 0'],axis=1)
print(Xtrain.shape)
Xtrain.head()

(3220100, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_12-18,order_HourGroup_18-24,order_HourGroup_06-12,order_HourGroup_00-06
0,1,0.092567,0.929434,1,1,0.1250,1,0,0,0
1,0,0.019529,0.929434,0,4,0.3750,1,0,0,0
2,1,0.043180,0.929434,1,2,0.1875,1,0,0,0
3,0,0.017755,0.929434,1,4,0.4375,1,0,0,0
4,1,0.018518,0.051992,1,4,0.2500,1,0,0,0


In [19]:

Xtest = X_test.copy()
Xtest = Xtest.drop(['Unnamed: 0'],axis=1)
print(Xtest.shape)
Xtest.head()

(402512, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_06-12,order_HourGroup_12-18,order_HourGroup_18-24,order_HourGroup_00-06
0,0,0.058997,0.929434,1,3,0.0625,1,0,0,0
1,1,0.017755,0.929434,1,2,0.1875,1,0,0,0
2,0,0.035028,0.929434,0,4,0.1875,0,1,0,0
3,1,0.037687,0.929434,0,1,0.2500,0,1,0,0
4,0,0.020333,0.929434,0,1,0.2500,0,1,0,0


In [19]:
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())


In [20]:
### We will create a table for comparing the results
#res = pd.DataFrame(columns=['model','accuracy', 'error'])
res = []

# sklearn.tree.DecisionTreeClassifier

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [24]:

mod_DecisionTree = LabelPowerset(DecisionTreeClassifier())


In [25]:
mod_DecisionTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       require_dense=[True, True])

In [26]:
# Make predictions 
pred_DecisionTree_test = mod_DecisionTree.predict(Xtest)

In [27]:
print(type(pred_DecisionTree_test))
print(pred_DecisionTree_test.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [28]:
pred_DecisionTree_test2 = np.array(pred_DecisionTree_test.todense())
pred_DecisionTree_test2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
#print(np.sum(pred2,axis=1))
print(np.sum(pred_DecisionTree_test2,axis=0))
print(pred_DecisionTree_test2.shape)

[ 7101  8319  8660  8310 37795 14156  7359 25711  7126 23446 11043 46136
 10831 13985 13372 13683  8774  7963  7338  9714  8270  7550  7591 10473
  7798 20859 15391 16824  7622  9312]
(402512, 30)


In [31]:
ham=hamming_score(ytest, pred_DecisionTree_test2)
ham

1.0

In [32]:
res.append({'model':'Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Decision Tree Classifier ', 'accuracy': 1.0}]

# sklearn.tree.ExtraTreeClassifier

In [33]:
from sklearn.tree import ExtraTreeClassifier

In [34]:
mod_ExtraTree = LabelPowerset(ExtraTreeClassifier())

In [35]:
mod_ExtraTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=ExtraTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'),
       require_dense=[True, True])

In [36]:
# Make predictions 
pred_ExtraTree_test = mod_ExtraTree.predict(Xtest)

In [37]:
print(type(pred_ExtraTree_test))
print(pred_ExtraTree_test.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [39]:
pred_ExtraTree_test2 = np.array(pred_ExtraTree_test.todense())
pred_ExtraTree_test2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [40]:
#print(np.sum(pred2,axis=1))
print(np.sum(pred_ExtraTree_test2,axis=0))
print(pred_ExtraTree_test2.shape)

[ 7101  8319  8662  8311 37795 14156  7360 25710  7127 23447 11044 46136
 10830 13985 13372 13683  8774  7957  7336  9713  8272  7551  7590 10474
  7798 20859 15391 16824  7622  9313]
(402512, 30)


In [41]:
ham=hamming_score(ytest,pred_ExtraTree_test2)
ham

0.9999279524585603

In [42]:
res.append({'model':'Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9999279524585603}]

# sklearn.ensemble.ExtraTreesClassifier

In [43]:
from sklearn.ensemble import ExtraTreesClassifier

C:\Users\malmagor\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [44]:
mod_ensemble_ExtraTree = LabelPowerset(ExtraTreeClassifier())

In [45]:
mod_ensemble_ExtraTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=ExtraTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'),
       require_dense=[True, True])

In [46]:
# predict
pred_ensemble_ExtraTree_test = mod_ensemble_ExtraTree.predict(Xtest)

In [47]:
print(type(pred_ensemble_ExtraTree_test))
print(pred_ensemble_ExtraTree_test.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [48]:
pred_ensemble_ExtraTree_test2 = np.array(pred_ensemble_ExtraTree_test.todense())
pred_ensemble_ExtraTree_test2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [49]:

print(np.sum(pred_ensemble_ExtraTree_test2,axis=0))
print(pred_ensemble_ExtraTree_test2.shape)

[ 7101  8320  8661  8310 37795 14157  7364 25711  7127 23446 11042 46136
 10831 13986 13376 13682  8774  7957  7333  9714  8273  7551  7591 10474
  7798 20859 15389 16825  7618  9311]
(402512, 30)


In [50]:
ham=hamming_score(ytest,pred_ensemble_ExtraTree_test2)
ham

0.9999006240807727

In [51]:
res.append({'model':'ensemble-Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9999279524585603},
 {'model': 'ensemble-Decision Tree Classifier ',
  'accuracy': 0.9999006240807727}]

# sklearn.ensemble.RandomForestClassifier

In [52]:
from sklearn.ensemble import RandomForestClassifier


In [53]:
# initialize label powerset multi-label classifier
mod_RandomForest = LabelPowerset(RandomForestClassifier())

In [54]:
mod_RandomForest.fit(Xtrain,ytrain)

LabelPowerset(classifier=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       require_dense=[True, True])

In [55]:
# predict
pred_RandomForest_test = mod_RandomForest.predict(Xtest)

In [56]:
print(type(pred_RandomForest_test))
print(pred_RandomForest_test.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [58]:
pred_RandomForest_test2 = np.array(pred_RandomForest_test.todense())
pred_RandomForest_test2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [59]:
print(np.sum(pred_ensemble_ExtraTree_test2,axis=0))
print(pred_ensemble_ExtraTree_test2.shape)

[ 7101  8320  8661  8310 37795 14157  7364 25711  7127 23446 11042 46136
 10831 13986 13376 13682  8774  7957  7333  9714  8273  7551  7591 10474
  7798 20859 15389 16825  7618  9311]
(402512, 30)


In [60]:
ham=hamming_score(ytest,pred_ensemble_ExtraTree_test2)
ham

0.9999006240807727

In [61]:
res.append({'model':'Random Forest ','accuracy':ham})
res

[{'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9999279524585603},
 {'model': 'ensemble-Decision Tree Classifier ',
  'accuracy': 0.9999006240807727},
 {'model': 'Random Forest ', 'accuracy': 0.9999006240807727}]

# sklearn.neighbors.KNeighborsClassifier

In [62]:
from sklearn.neighbors import KNeighborsClassifier

In [63]:
# initialize label powerset multi-label classifier
mod_KNeighbors = LabelPowerset(KNeighborsClassifier(n_neighbors=3))

In [64]:
mod_KNeighbors.fit(Xtrain,ytrain)

LabelPowerset(classifier=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'),
       require_dense=[True, True])

In [65]:
# predict
pred_KNeighbors_test = mod_KNeighbors.predict(Xtest)

In [66]:
print(type(pred_KNeighbors_test))
print(pred_KNeighbors_test.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [67]:
pred_KNeighbors_test2 = np.array(pred_KNeighbors_test.todense())
pred_KNeighbors_test2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [68]:
print(np.sum(pred_KNeighbors_test2,axis=0))
print(pred_KNeighbors_test2.shape)

[ 7103  8319  8663  8331 37796 14157  7353 25708  7141 23447 11040 46136
 10841 13991 13379 13675  8768  7950  7336  9710  8270  7543  7591 10478
  7808 20861 15380 16818  7615  9304]
(402512, 30)


In [69]:
ham=hamming_score(ytest,pred_KNeighbors_test2)
ham

0.9994236196684819